In [ ]:
import xarray as xr
import pandas as pd
import os
xar = xr.open_dataset("data/era5_wind_1980_01.nc")

xar.data_vars


ValueError: found the following matches with the input file in xarray's IO backends: ['netcdf4', 'h5netcdf']. But their dependencies may not be installed, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

In [2]:
xar['valid_time'][0].values

np.datetime64('1980-01-01T00:00:00.000000000')

In [3]:
xar.to_array

<bound method Dataset.to_array of <xarray.Dataset> Size: 74MB
Dimensions:     (valid_time: 744, latitude: 77, longitude: 81)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 6kB 1980-01-01 ... 1980-01-31T23:...
  * latitude    (latitude) float64 616B 14.0 13.75 13.5 ... -4.5 -4.75 -5.0
  * longitude   (longitude) float64 648B -83.0 -82.75 -82.5 ... -63.25 -63.0
    expver      (valid_time) <U4 12kB ...
Data variables:
    u10         (valid_time, latitude, longitude) float32 19MB ...
    v10         (valid_time, latitude, longitude) float32 19MB ...
    u100        (valid_time, latitude, longitude) float32 19MB ...
    v100        (valid_time, latitude, longitude) float32 19MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts


In [4]:
training = xar['v10'][:,13,74].to_numpy()
label = xar['v10'][:,14,74].to_numpy()

In [5]:
training = xar['v10'][:,13,74].to_numpy()
label = xar['v10'][:,14,74].to_numpy()

In [6]:
xar.sel(
    latitude=14,
    longitude=-74,
    method='nearest'
).to_dataframe().reset_index().drop(['number','expver'], axis =1)

,valid_time,latitude,longitude,u10,v10,u100,v100
0,1980-01-01 00:00:00,14.0,-74.0,-9.437561,-3.510574,-10.920517,-4.021698
1,1980-01-01 01:00:00,14.0,-74.0,-9.492294,-3.362610,-11.023361,-3.871246
2,1980-01-01 02:00:00,14.0,-74.0,-9.540619,-2.798782,-11.112534,-3.237015
3,1980-01-01 03:00:00,14.0,-74.0,-9.413589,-2.430038,-10.971603,-2.797546
4,1980-01-01 04:00:00,14.0,-74.0,-9.409821,-1.910248,-10.915482,-2.184509
...,...,...,...,...,...,...,...
739,1980-01-31 19:00:00,14.0,-74.0,-8.230865,-2.228455,-9.287674,-2.473129
740,1980-01-31 20:00:00,14.0,-74.0,-8.281479,-2.497833,-9.360565,-2.787949
741,1980-01-31 21:00:00,14.0,-74.0,-8.480927,-2.561142,-9.616287,-2.874390
742,1980-01-31 22:00:00,14.0,-74.0,-8.533646,-2.197769,-9.671906,-2.445251


In [7]:
path = 'data/'
files = os.listdir(path)
# Solo se retienen archivos nc
dict_files = {
    'date':list(),
    'link':list()
}

for file in files:
    if '.nc' in file:
        # generacion link
        link = os.path.join(
            path,
            file
        )
        #Formateo fecha
        year = file.split('_')[2]
        month = file.split('_')[3].split('.')[0]
        date = pd.to_datetime(year + '/' + month)
        # Almacenamiento
        dict_files['date'].append(date)
        dict_files['link'].append(link)
# Creacion del dataframe
df_files = pd.DataFrame(dict_files)
df_files.set_index(
    'date',
    drop=True,
    inplace=True
)
df_files = df_files.sort_index()

In [ ]:
combined = xr.Dataset()

latitude = 16
longitude = -74

for i,link in enumerate(df_files['link'].to_numpy()):
    if(i>10):break
    temporal = xr.open_dataset(link).sel(
        latitude=latitude,
        longitude=longitude,
        method='nearest'
    )
    combined = xr.concat([combined,temporal],dim = "time")

RuntimeError: NetCDF: HDF error

In [11]:
combined.to_dataframe().reset_index()

,time,valid_time,u10,v10,u100,v100,number,latitude,longitude,expver
0,0,1980-01-01 00:00:00,NaN,NaN,NaN,NaN,0,14.0,-74.0,0001
1,0,1980-01-01 01:00:00,NaN,NaN,NaN,NaN,0,14.0,-74.0,0001
2,0,1980-01-01 02:00:00,NaN,NaN,NaN,NaN,0,14.0,-74.0,0001
3,0,1980-01-01 03:00:00,NaN,NaN,NaN,NaN,0,14.0,-74.0,0001
4,0,1980-01-01 04:00:00,NaN,NaN,NaN,NaN,0,14.0,-74.0,0001
...,...,...,...,...,...,...,...,...,...,...
96475,11,1980-11-30 19:00:00,-9.007050,-1.832840,-10.338379,-2.051880,0,14.0,-74.0,0001
96476,11,1980-11-30 20:00:00,-8.852646,-2.241882,-10.145172,-2.521225,0,14.0,-74.0,0001
96477,11,1980-11-30 21:00:00,-8.777252,-2.800812,-10.055450,-3.162888,0,14.0,-74.0,0001
96478,11,1980-11-30 22:00:00,-8.643341,-2.607437,-9.897675,-2.925491,0,14.0,-74.0,0001
